In [1]:
%run TF_StateFarm.py

read train images


In [2]:
#train_data = train_data.astype('float16')

In [3]:
IMAGE_PIXELS

50176

In [19]:
# Generate the TF Graph
image_width = 28
image_height = 28
x_placeholder = tf.placeholder('float', shape=[None, image_width])
image = tf.reshape(x_placeholder, [-1,image_width , image_height,1])
y_placeholder = tf.placeholder('float', shape=[None, NUM_CLASSES])

In [5]:
from NN_Components import *
# build model
# input is the num hidden and x_placeholder, y_placeholder
# fixed vars
def graph_object( images ):
    
    #pad_1 = ZeroPadding(data=images)
    #conv_1 = conv_layer(input_tensor = pad_1, patch_size=3, channels=1, output_dim=64, layer_name = 'conv 1')
    #pad_2 = ZeroPadding(data=conv_1)
    #conv_2 = conv_layer(input_tensor = pad_2, patch_size=3, channels=1, output_dim=64, layer_name = 'conv 2')
    #pool_1 = max_pool_2x2(conv_2)
    pass

def loss( ):
    pass

In [6]:
LEARNING_RATE = 1e-4
epochs_completed = 0
index_in_epoch = 0
num_examples = train_data.shape[0]

DROPOUT = 0.5
BATCH_SIZE = 50

In [8]:
## run tf
conv_1 = conv_layer(input_tensor = image, patch_size=5, 
                    channels=1, output_dim=32, layer_name = 'conv_1')
print conv_1.get_shape()
pool_1 = max_pool_2x2(conv_1)
print pool_1.get_shape()

conv_2 = conv_layer(input_tensor = pool_1, patch_size=5, 
                    channels=32, output_dim=64, layer_name = 'conv_2')
print conv_2.get_shape()
pool_2 = max_pool_2x2(conv_2)
print pool_2.get_shape()


# Fully Connected Layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# (40000, 7, 7, 64) => (40000, 3136)
h_pool2_flat = tf.reshape(pool_2, [-1, 7*7*64])
print h_pool2_flat.get_shape()
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# drop out
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer for deep net
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entropy = -tf.reduce_sum(y_placeholder*tf.log(y))


# optimisation function
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# evaluation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_placeholder,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
# To predict values from test data, highest probability is picked from "one-hot vector" indicating that chances of  an image being one of the digits are highest.
# prediction function
#[0.1, 0.9, 0.2, 0.1, 0.1 0.3, 0.5, 0.1, 0.2, 0.3] => 1
predict = tf.argmax(y,1)

(?, 14, 14, 32)
(?, 7, 7, 32)
(?, 4, 4, 64)
(?, 2, 2, 64)
(?, 3136)


In [10]:
import numpy as np

In [ ]:
import NN_Components
reload(NN_Components)
#%load_ext autoreload
%autoreload 2

In [11]:
# serve data by batches
def next_batch(batch_size):
    
    global train_data
    global train_target
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_data[perm]
        train_labels = train_target[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_data[start:end], train_target[start:end]
# Now when all operations for every variable are defined in TensorFlow graph all computations will be performed outside Python environment.
# start TensorFlow session
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

sess.run(init)

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f44a4a27a10>> ignored


In [22]:
TRAINING_ITERATIONS = 5
DROPOUT = 0.9
x = x_placeholder
y_ = y_placeholder

In [23]:
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(BATCH_SIZE)        

    # check progress on every 1st,2nd,...,10th,20th,...,100th... step
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       
        #if(VALIDATION_SIZE):
        #    validation_accuracy = accuracy.eval(feed_dict={ x: validation_images[0:BATCH_SIZE], 
        #                                                    y_: validation_labels[0:BATCH_SIZE], 
        #                                                    keep_prob: 1.0})                                  
        #    print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
        #    
        #    validation_accuracies.append(validation_accuracy)
            
        #else:
        print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # increase display_step
        if i%(display_step*10) == 0 and i:
            display_step *= 10
    # train on batch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

ValueError: Cannot feed value of shape (50, 224, 224, 1) for Tensor u'Placeholder_5:0', which has shape '(?, 28)'